# Pytorch로 RNN 구현하기
참고 : https://justkode.kr/deep-learning/pytorch-rnn

- input_size : Input의 사이즈에 해당하는 수를 입력
- hidden_size : 은닉층의 사이즈에 해당하는 수를 입력
- num_layers : RNN의 은닉층 레이어 갯수. (기본값은 1)
- nonlinearity : 비선형 활성화 함수를 선택 tanh, relu 중 선택 가능 (기본값은 tanh)
- bias : 바이어스 값 활성화 여부를 선택 (기본값은 True)
- batch_first : True일 시, output 값의 사이즈는 (batch, seq, feature) (기본값은 False)
- dropout : 드롭아웃 비율을 설정 (기본값은 0)
- bidirectional : True일 시 양방향 RNN (기본값은 False)

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk",
            "you hate coffee"]
dtype = torch.float

## Word processing

In [2]:
word_list = list(set(" ".join(sentences).split()))
print(f"word_list = {word_list}")
word_dict = {w: i for i, w in enumerate(word_list)}
print(f"word_dict = {word_dict}")
number_dict = {i: w for i, w in enumerate(word_list)}
print(f"number_dict = {number_dict}")
n_class = len(word_dict)
print(f"n_class = {n_class}")

word_list = ['i', 'coffee', 'like', 'cat', 'hate', 'milk', 'love', 'you', 'dog']
word_dict = {'i': 0, 'coffee': 1, 'like': 2, 'cat': 3, 'hate': 4, 'milk': 5, 'love': 6, 'you': 7, 'dog': 8}
number_dict = {0: 'i', 1: 'coffee', 2: 'like', 3: 'cat', 4: 'hate', 5: 'milk', 6: 'love', 7: 'you', 8: 'dog'}
n_class = 9


## TextRNN Parameter

In [3]:
batch_size = len(sentences)
n_step = 2 #학습하려고 하는 문장의 길이 -1
n_hidden = 5 #은닉층 사이즈

def make_batch(sentences):
    input_batch = []
    target_batch = []
    
    for sen in sentences:
        word = sen.split()
        input = [word_dict[n] for n in word[:-1]]
        target = word_dict[word[-1]]
        
        input_batch.append(np.eye(n_class)[input]) #one-hot encoding
        target_batch.append(target)
        
    return input_batch, target_batch

input_batch, target_batch = make_batch(sentences)
print(f"input_batch, target_batch = {input_batch, target_batch}")
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
print(f"input_batch = {input_batch}")
target_batch = torch.tensor(target_batch, dtype=torch.int64)
print(f"target_batch = {target_batch}")

input_batch, target_batch = ([array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.]]), array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]]), array([[1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.]]), array([[0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]])], [8, 1, 5, 3, 5, 1])
input_batch = tensor([[[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[1., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 0., 0., 1., 0.],
         [0., 0., 1., 0., 0., 0., 0., 0., 0.]],


<ipython-input-3-257c8e02e397>:21: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)


## TextRNN

In [4]:
class TextRNN(nn.Module):
    def __init__(self):
        super(TextRNN, self).__init__()
        
        self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
        self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
        self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
        self.Softmax = nn.Softmax(dim=1)
        
    def forward(self, hidden, X):
        X = X.transpose(0, 1)
        outputs, hidden = self.rnn(X, hidden)
        outputs = outputs[-1] #최종 예측 hidden Layer
        model = torch.mm(outputs, self.W) + self.b #최종 예측 최종 출력층
        return model

## Training

In [5]:
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
    hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
    output = model(hidden, input_batch)
    loss = criterion(output, target_batch)
    
    if (epoch + 1) % 100 == 0:
        print('Epoch: ', '%04d'%(epoch + 1), 'cost = ', '{:.6f}'.format(loss))
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
input = [sen.split()[:2] for sen in sentences]

hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

/usr/local/lib/python3.8/dist-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Epoch:  0100 cost =  0.430150
Epoch:  0200 cost =  0.065829
Epoch:  0300 cost =  0.024365
Epoch:  0400 cost =  0.013674
Epoch:  0500 cost =  0.009024
[['i', 'like'], ['i', 'love'], ['i', 'hate'], ['you', 'like'], ['you', 'love'], ['you', 'hate']] -> ['dog', 'coffee', 'milk', 'cat', 'milk', 'coffee']
